In [1]:
import sys
sys.path.append('..')

from utilities import *

warnings.filterwarnings('ignore')

raw_data = pd.read_excel("../data/Datos_Market_copy.xlsx")

sa = SalesAnalysis(raw_data)

data = sa.data[sa.brand35]

train_data, test_data = sa.divide_data_for_train_and_test(data)

/Users/oscarjimenezbou/Documents/TFG_ADE/code/TFG/lib/python3.9/site-packages/urllib3/__init__.py:35: NotOpenSSLWarning: urllib3 v2 only supports OpenSSL 1.1.1+, currently the 'ssl' module is compiled with 'LibreSSL 2.8.3'. See: https://github.com/urllib3/urllib3/issues/3020
  warnings.warn(


In [2]:
model, design_info, selected_columns = sa.modelization_with_backward_elimination(train_data)


In [3]:
'''Incorrect way of using auto_arima with exogenous variables'''

# auto_arima_model = auto_arima( 
#     y=train_data['volume.sales'],
#     X=train_data[selected_columns],
#     start_p=0,
#     d=1,  
#     start_q=0,
#     max_p=3, 
#     max_q=3,
#     start_P=0,
#     D=1,  # Set seasonal differencing explicitly
#     start_Q=0,
#     max_P=1, 
#     max_Q=1,
#     m=12,  # Monthly data
#     seasonal=True,
#     trace=True,
#     error_action="warn",  # Change to warn to see potential issues
#     suppress_warnings=True,
#     stepwise=True,
#     random_state=42,
#     n_fits=50,
#     information_criterion='aic'
# )

'Incorrect way of using auto_arima with exogenous variables'

#### Create the variable X_train_exog with the selected columns from the regression model

Recreate interactions of all variables

In [4]:
# Create the exogenous variables matrix for auto_arima
# We need to recreate the design matrix using the same transformations

# First, rename columns to match the formula used in modelization_with_backward_elimination
train_data_for_patsy = train_data.copy()
train_data_for_patsy.rename(
    columns={
        "value.sales": "value_sales",
        "unit.sales": "unit_sales", 
        "volume.sales": "volume_sales",
        "pack.size": "pack_size",
    },
    inplace=True,
)

# Use the same formula as in the regression
formula = "volume_sales ~ (price + C(supermarket) + C(variant) + C(pack_size)) ** 2"

# Create the design matrix
y_design, X_design = patsy.dmatrices(formula, data=train_data_for_patsy, return_type="dataframe")

print("Design matrix shape:", X_design.shape)
print("Design matrix columns:", X_design.columns.tolist())

Design matrix shape: (644, 47)
Design matrix columns: ['Intercept', 'C(supermarket)[T.supermarket-B]', 'C(supermarket)[T.supermarket-C]', 'C(supermarket)[T.supermarket-D]', 'C(variant)[T.light]', 'C(variant)[T.standard]', 'C(variant)[T.vegan]', 'C(pack_size)[T.351 - 500 GR]', 'C(pack_size)[T.501 - 700 GR]', 'C(pack_size)[T.701 - 1000 GR]', 'C(supermarket)[T.supermarket-B]:C(variant)[T.light]', 'C(supermarket)[T.supermarket-C]:C(variant)[T.light]', 'C(supermarket)[T.supermarket-D]:C(variant)[T.light]', 'C(supermarket)[T.supermarket-B]:C(variant)[T.standard]', 'C(supermarket)[T.supermarket-C]:C(variant)[T.standard]', 'C(supermarket)[T.supermarket-D]:C(variant)[T.standard]', 'C(supermarket)[T.supermarket-B]:C(variant)[T.vegan]', 'C(supermarket)[T.supermarket-C]:C(variant)[T.vegan]', 'C(supermarket)[T.supermarket-D]:C(variant)[T.vegan]', 'C(supermarket)[T.supermarket-B]:C(pack_size)[T.351 - 500 GR]', 'C(supermarket)[T.supermarket-C]:C(pack_size)[T.351 - 500 GR]', 'C(supermarket)[T.supermar

Only select those that were selected in the regression model

In [5]:
# Filter the design matrix to include only the selected columns from backward elimination
# Remove 'Intercept' since auto_arima handles that automatically
selected_columns_no_intercept = [col for col in selected_columns if col != 'Intercept']

print("Selected columns without intercept:", selected_columns_no_intercept)
print("Number of exogenous variables:", len(selected_columns_no_intercept))

# Create the exogenous variables matrix for training
X_train_exog = X_design[selected_columns_no_intercept]

print("Exogenous variables matrix shape:", X_train_exog.shape)
print("Exogenous variables matrix columns:", X_train_exog.columns.tolist())

Selected columns without intercept: ['C(supermarket)[T.supermarket-B]', 'C(supermarket)[T.supermarket-D]', 'C(variant)[T.standard]', 'C(variant)[T.vegan]', 'C(pack_size)[T.351 - 500 GR]', 'C(pack_size)[T.501 - 700 GR]', 'C(pack_size)[T.701 - 1000 GR]', 'C(supermarket)[T.supermarket-B]:C(variant)[T.light]', 'C(supermarket)[T.supermarket-C]:C(variant)[T.light]', 'C(supermarket)[T.supermarket-B]:C(variant)[T.standard]', 'C(supermarket)[T.supermarket-C]:C(variant)[T.standard]', 'C(supermarket)[T.supermarket-D]:C(variant)[T.standard]', 'C(supermarket)[T.supermarket-B]:C(variant)[T.vegan]', 'C(supermarket)[T.supermarket-C]:C(pack_size)[T.351 - 500 GR]', 'C(supermarket)[T.supermarket-D]:C(pack_size)[T.351 - 500 GR]', 'C(supermarket)[T.supermarket-D]:C(pack_size)[T.501 - 700 GR]', 'C(supermarket)[T.supermarket-B]:C(pack_size)[T.701 - 1000 GR]', 'C(variant)[T.light]:C(pack_size)[T.351 - 500 GR]', 'C(variant)[T.vegan]:C(pack_size)[T.351 - 500 GR]', 'C(variant)[T.light]:C(pack_size)[T.501 - 700 G

Comparison between model features and X_train_exog features - Do they match? Yes

In [6]:
# Print the regression model coefficients and their names
print("Regression model coefficients:")
print("=" * 50)
for i, (coef_name, coef_value) in enumerate(zip(model.params.index, model.params.values)):
    print(f"{i+1:2d}. {coef_name:<50} {coef_value:>12.6f}")

print("\n\nX_train_exog columns:")
print("=" * 50)
for i, col_name in enumerate(X_train_exog.columns):
    print(f"{i+1:2d}. {col_name}")

print("\n\nComparison - Do they match?")
print("=" * 50)
model_features = list(model.params.index)
exog_features = list(X_train_exog.columns)

# Remove 'Intercept' from model features for comparison since X_train_exog doesn't have it
model_features_no_intercept = [f for f in model_features if f != 'Intercept']

print(f"Model features (excluding Intercept): {len(model_features_no_intercept)}")
print(f"X_train_exog features: {len(exog_features)}")

if set(model_features_no_intercept) == set(exog_features):
    print("✅ YES - All features match perfectly!")
else:
    print("❌ NO - Features don't match")
    
    missing_in_exog = set(model_features_no_intercept) - set(exog_features)
    missing_in_model = set(exog_features) - set(model_features_no_intercept)
    
    if missing_in_exog:
        print(f"\nFeatures in model but missing in X_train_exog: {len(missing_in_exog)}")
        for feature in missing_in_exog:
            print(f"  - {feature}")
    
    if missing_in_model:
        print(f"\nFeatures in X_train_exog but missing in model: {len(missing_in_model)}")
        for feature in missing_in_model:
            print(f"  - {feature}")

Regression model coefficients:
 1. Intercept                                          12101.529434
 2. C(supermarket)[T.supermarket-B]                    23186.891702
 3. C(supermarket)[T.supermarket-D]                    -105090.039176
 4. C(variant)[T.standard]                             26242.235474
 5. C(variant)[T.vegan]                                75206.390835
 6. C(pack_size)[T.351 - 500 GR]                       82197.774832
 7. C(pack_size)[T.501 - 700 GR]                       21792.130242
 8. C(pack_size)[T.701 - 1000 GR]                      -19934.073669
 9. C(supermarket)[T.supermarket-B]:C(variant)[T.light] -19239.020609
10. C(supermarket)[T.supermarket-C]:C(variant)[T.light] 30044.882830
11. C(supermarket)[T.supermarket-B]:C(variant)[T.standard] -52039.040199
12. C(supermarket)[T.supermarket-C]:C(variant)[T.standard] -32155.935126
13. C(supermarket)[T.supermarket-D]:C(variant)[T.standard] 69131.249607
14. C(supermarket)[T.supermarket-B]:C(variant)[T.vegan] 71122.920

#### Run Auto ARIMA model with exogenous variables

In [7]:
# Now we can run auto_arima with the correct exogenous variables
auto_arima_model = auto_arima( 
    y=train_data['volume.sales'],
    X=X_train_exog,  # Use the properly formatted exogenous variables
    start_p=0,
    d=1,  
    start_q=0,
    max_p=3, 
    max_q=3,
    start_P=0,
    D=1,  # Set seasonal differencing explicitly
    start_Q=0,
    max_P=1, 
    max_Q=1,
    m=12,  # Monthly data
    seasonal=True,
    trace=True,
    error_action="warn",  # Change to warn to see potential issues
    suppress_warnings=True,
    stepwise=True,
    random_state=42,
    n_fits=50,
    information_criterion='aic'
)

print("Auto ARIMA model fitted successfully!")
print(auto_arima_model.summary())

Performing stepwise search to minimize aic
 ARIMA(0,1,0)(0,1,0)[12]             : AIC=18700.911, Time=0.40 sec
 ARIMA(1,1,0)(1,1,0)[12]             : AIC=14983.257, Time=19.09 sec
 ARIMA(0,1,1)(0,1,1)[12]             : AIC=inf, Time=151.66 sec
 ARIMA(1,1,0)(0,1,0)[12]             : AIC=15113.277, Time=0.56 sec
 ARIMA(1,1,0)(1,1,1)[12]             : AIC=inf, Time=60.61 sec
 ARIMA(1,1,0)(0,1,1)[12]             : AIC=inf, Time=44.92 sec
 ARIMA(0,1,0)(1,1,0)[12]             : AIC=15113.200, Time=16.41 sec
 ARIMA(2,1,0)(1,1,0)[12]             : AIC=14940.126, Time=21.35 sec
 ARIMA(2,1,0)(0,1,0)[12]             : AIC=15067.640, Time=1.50 sec
 ARIMA(2,1,0)(1,1,1)[12]             : AIC=inf, Time=63.86 sec
 ARIMA(2,1,0)(0,1,1)[12]             : AIC=inf, Time=54.68 sec
 ARIMA(3,1,0)(1,1,0)[12]             : AIC=14902.971, Time=41.99 sec
 ARIMA(3,1,0)(0,1,0)[12]             : AIC=15033.538, Time=2.07 sec
 ARIMA(3,1,0)(1,1,1)[12]             : AIC=inf, Time=51.23 sec
 ARIMA(3,1,0)(0,1,1)[12]      

#### Filter series for forecasting

In [8]:
filter_series_train_data = train_data[
    (train_data["supermarket"] == "supermarket-A")
    & (train_data["variant"] == "standard")
    & (train_data["pack.size"] == "351 - 500 GR")
]

filter_series_test_data = test_data[
    (test_data["supermarket"] == "supermarket-A")
    & (test_data["variant"] == "standard")
    & (test_data["pack.size"] == "351 - 500 GR")
]

#### Forecast with exogenous variables

In [9]:
#TODO: this is solution GPT suggested me, to forecast with exogenous variables. Check it later. I have to adapt it to my code. 
# It is the raw answer from GPT. 


# === 0) Supongo que ya tienes esto hecho antes ===
# raw_data, sa = SalesAnalysis(...), data = sa.data[sa.brand35]
# train_data, test_data = sa.divide_data_for_train_and_test(data)
# model, design_info, selected_columns = sa.modelization_with_backward_elimination(train_data)

# Re-creas la misma matriz X para TRAIN con patsy (ya lo tienes):
train_p = train_data.copy()
train_p = train_p.rename(columns={"value.sales":"value_sales","unit.sales":"unit_sales",
                                  "volume.sales":"volume_sales","pack.size":"pack_size"})
formula = "volume_sales ~ (price + C(supermarket) + C(variant) + C(pack_size)) ** 2"
y_design, X_design = patsy.dmatrices(formula, data=train_p, return_type="dataframe")

selected_cols_no_intercept = [c for c in selected_columns if c != "Intercept"]
X_train_exog = X_design[selected_cols_no_intercept]
y_train_global = train_data['volume.sales']

# === 1) Modelo ARIMAX global solo para escoger órdenes (ya lo tienes hecho) ===
# auto_arima_model = auto_arima(..., y=y_train_global, X=X_train_exog, m=..., seasonal=True, ...)
order_glob = auto_arima_model.order
seasonal_order_glob = auto_arima_model.seasonal_order
print("Órdenes globales:", order_glob, seasonal_order_glob)

# === 2) Función para construir X_exog con las MISMAS columnas (y orden) para cualquier df ===
def build_exog_from_design(df_like_train, design_info, selected_cols_no_intercept):
    # Renombrar como en TRAIN
    dfp = df_like_train.rename(columns={"value.sales":"value_sales","unit.sales":"unit_sales",
                                        "volume.sales":"volume_sales","pack.size":"pack_size"}).copy()
    # Usar el MISMO design_info para garantizar mismas dummies/contrastes
    X_new = patsy.build_design_matrices([design_info], dfp, return_type="dataframe")[0]
    # Asegurar columnas (si falta alguna, crearla con 0; si sobra, descartar)
    for c in selected_cols_no_intercept:
        if c not in X_new.columns:
            X_new[c] = 0.0
    X_new = X_new[selected_cols_no_intercept]  # mismo orden
    return X_new

# === 3) Forecast por serie filtrada, re-ajustando SARIMAX con ÓRDENES GLOBALES ===
from statsmodels.tsa.statespace.sarimax import SARIMAX
from sklearn.metrics import mean_absolute_error, mean_squared_error
import numpy as np

def forecast_series_with_global_structure(train_df, test_df,
                                          order_glob, seasonal_order_glob,
                                          design_info, selected_cols_no_intercept):
    # Endógena
    y_tr = train_df['volume.sales']
    y_te = test_df['volume.sales']

    # Exógenas con el MISMO pipeline que el global
    X_tr = build_exog_from_design(train_df, design_info, selected_cols_no_intercept)
    X_te = build_exog_from_design(test_df,   design_info, selected_cols_no_intercept)

    # (Opcional) reducir a las exógenas que realmente varían en la serie:
    varying = [c for c in X_tr.columns if X_tr[c].std() > 0 or X_te[c].std() > 0]
    # Evitar quedarse sin columnas (p. ej. si sólo varía 'price'):
    if len(varying) == 0:
        X_tr_use = None
        X_te_use = None
    else:
        X_tr_use = X_tr[varying]
        X_te_use = X_te[varying]

    # Ajuste rápido con órdenes globales (no hay búsqueda)
    model_loc = SARIMAX(endog=y_tr,
                        exog=X_tr_use,
                        order=order_glob,
                        seasonal_order=seasonal_order_glob,
                        enforce_stationarity=False,
                        enforce_invertibility=False)
    res_loc = model_loc.fit(disp=False)

    # Forecast sobre TEST
    steps = len(test_df)
    pred_res = res_loc.get_forecast(steps=steps, exog=X_te_use)
    y_pred = pred_res.predicted_mean
    ci = pred_res.conf_int()

    # Métricas
    mae  = mean_absolute_error(y_te, y_pred)
    rmse = mean_squared_error(y_te, y_pred, squared=False)

    return y_pred, ci, mae, rmse, res_loc

# === 4) Ejemplo: misma serie que ya filtraste ===
ftrain = filter_series_train_data.copy()
ftest  = filter_series_test_data.copy()

# (Asegura índice fecha coherente y misma freq en ambos si vas a usar estacionalidad)
for df_ in (ftrain, ftest):
    df_['date'] = pd.to_datetime(df_['date'])
    df_.sort_values('date', inplace=True)
    df_.set_index('date', inplace=True)
    # df_ = df_.asfreq('MS')  # <- si es mensual; ajusta a tu caso

y_pred, ci, mae, rmse, fitted_local = forecast_series_with_global_structure(
    ftrain.reset_index(),  # la función espera columnas (no índice) para patsy
    ftest.reset_index(),
    order_glob, seasonal_order_glob,
    design_info, selected_cols_no_intercept
)

print(f"MAE={mae:.3f}  RMSE={rmse:.3f}")
# (Opcional) Plot rápido
import matplotlib.pyplot as plt
plt.figure()
ftest['volume.sales'].plot(label='test')
y_pred.index = ftest.index  # alinear fechas
y_pred.plot(label='forecast')
plt.fill_between(ci.index, ci.iloc[:,0], ci.iloc[:,1], alpha=0.2)
plt.legend(); plt.title('Forecast con estructura global (órdenes y xreg)'); plt.show()


Órdenes globales: (3, 1, 0) (1, 1, 0, 12)


TypeError: got an unexpected keyword argument 'squared'